In [1]:
import pickle
import tensorflow as tf
from tensorflow.keras import models

import os
import numpy as np
import time
import pandas as pd

2022-07-28 13:44:30.389394: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
def load_data(filename):
    data = []
    gene_names = []
    lines = open(filename).readlines()
    sample_names = lines[0].replace('\n', '').split('\t')[1:]
    dx = 1

    for line in lines[dx:]:
        values = line.replace('\n', '').split('\t')
        gene = str.upper(values[0])
        gene_names.append(gene)
        data.append(values[1:])
    data = np.array(data, dtype='float32')
    data = np.transpose(data)

    return data, sample_names, gene_names

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
    
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [3]:
print(os.getcwd())
os.chdir("/home/wmbio/WORK/gitworking/Dependency_prediction/")
print(os.getcwd())

/home/wmbio/WORK/gitworking/Dependency_prediction/prediction/src
/home/wmbio/WORK/gitworking/Dependency_prediction


In [4]:
with tf.device('/cpu:0'):
    model_name = "model_ccl501_exp_0728"
    model_saved = models.load_model("prediction/custom_model/%s.h5" % model_name, 
                                    custom_objects={"coeff_determination": coeff_determination})

    # model_name = "model_exp_paper"
    # model_saved = models.load_model("prediction/data/full_results_models_paper/models/%s.h5" % model_name)
# model_paper is the full 4-omics DeepDEP model used in the paper
# user can choose from single-omics, 2-omics, or full DeepDEP models from the
# /data/full_results_models_paper/models/ directory

2022-07-28 13:45:19.428825: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-28 13:45:19.429297: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-28 13:45:19.460563: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-28 13:45:19.461032: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.807GHz coreCount: 28 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 335.32GiB/s
2022-07-28 13:45:19.461063: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-28 13:45:19.463917: I tensorflow/stream_executor/platform/d

* **prediction**

In [5]:
data_mut, sample_names_mut, gene_names_mut = load_data(
    "prediction/data/prediction_data/predict_mut_prediction.txt")
data_exp, sample_names_exp, gene_names_exp = load_data(
    "prediction/data/prediction_data/predict_exp_prediction.txt")
data_cna, sample_names_cna, gene_names_cna = load_data(
    "prediction/data/prediction_data/predict_cna_prediction.txt")
data_meth, sample_names_meth, gene_names_meth = load_data(
    "prediction/data/prediction_data/predict_meth_prediction.txt")
data_fprint_DepOIs, gene_names_fprint, function_names_fprint = load_data(
    "prediction/data/prediction_data/predict_fingerprint_prediction.txt")
print("\n\nDatasets successfully loaded.\n\n")



Datasets successfully loaded.




In [6]:
batch_size = 500
first_to_predict = 10
# predict the first 10 samples for DEMO ONLY, for all samples please substitute 10 by data_mut_tcga.shape[0]
# prediction results of all 8238 TCGA samples can be found in /data/full_results_models_paper/predictions/

In [7]:
with tf.device('/cpu:0'):
    t = time.time()
    data_pred = np.zeros((first_to_predict, data_fprint_DepOIs.shape[0]))
    for z in np.arange(0, first_to_predict):
        data_pred_tmp = model_saved.predict([
            # data_mut[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_exp[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_cna[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_meth[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_fprint_DepOIs
        ], 
            batch_size=batch_size, verbose=0)
        data_pred[z] = np.transpose(data_pred_tmp)
        print("Unscreended sample %d predicted..." % z)

2022-07-28 14:02:37.418547: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-07-28 14:02:37.437167: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3187200000 Hz


Unscreended sample 0 predicted...
Unscreended sample 1 predicted...
Unscreended sample 2 predicted...
Unscreended sample 3 predicted...
Unscreended sample 4 predicted...
Unscreended sample 5 predicted...
Unscreended sample 6 predicted...
Unscreended sample 7 predicted...
Unscreended sample 8 predicted...
Unscreended sample 9 predicted...


In [8]:
# write prediction results to txt
data_pred_df = pd.DataFrame(data=np.transpose(data_pred), index=gene_names_fprint, columns=sample_names_mut[0:first_to_predict])

In [9]:
data_pred_df

,SW837,COLO678,PECAPJ15,NCIH211,HUH7,NCIH1836,BHY,P31FUJ,GAMG,SKNAS
AARS2,-0.408114,-0.391395,-0.368480,-0.423975,-0.371855,-0.431483,-0.374573,-0.403777,-0.423936,-0.406856
AASDHPPT,-0.308414,-0.296563,-0.288115,-0.323961,-0.287482,-0.329810,-0.287008,-0.304365,-0.323926,-0.307162
ABCE1,-1.835570,-1.857282,-1.888475,-1.814494,-1.883631,-1.803071,-1.880203,-1.841177,-1.814557,-1.836815
ABCG1,-0.182859,-0.179189,-0.187012,-0.190023,-0.186454,-0.192433,-0.185426,-0.179919,-0.190009,-0.181994
ABHD11,-0.482753,-0.470009,-0.439564,-0.495434,-0.445417,-0.500681,-0.450117,-0.479503,-0.495406,-0.481810
...,...,...,...,...,...,...,...,...,...,...
ZNF384,-0.120732,-0.117148,-0.111664,-0.125219,-0.112616,-0.128158,-0.113383,-0.119785,-0.125204,-0.120469
ZNF407,-0.868000,-0.848570,-0.818520,-0.883583,-0.822836,-0.890602,-0.826385,-0.863426,-0.883545,-0.866673
ZNF521,-0.214702,-0.207518,-0.205017,-0.216842,-0.205677,-0.217927,-0.206499,-0.212860,-0.216836,-0.214172
ZNHIT2,-1.867973,-1.852215,-1.824394,-1.888821,-1.827432,-1.898038,-1.829879,-1.862843,-1.888773,-1.866474


In [ ]:
# write prediction results to txt
data_pred_df = pd.DataFrame(data=np.transpose(data_pred), index=gene_names_fprint, columns=sample_names_mut[0:first_to_predict])

In [ ]:
data_pred_df

In [10]:
data_pred_df.to_csv('custom_exp.csv')

In [ ]:
# pd.DataFrame.to_csv(data_pred_df, path_or_buf="/results/predictions/tcga_predicted_data_%s_demo.txt" % model_name, sep='\t', index_label='CRISPR_GENE', float_format='%.4f')
# print("\n\nPrediction completed in %.1f mins.\nResults saved in /results/predictions/tcga_predicted_data_%s_demo.txt\n\n" % (
#     (time.time()-t)/60, model_name))